# Análise de Dados de Oficina Mecânica

Este projeto tem como objetivo realizar uma análise dos dados de uma oficina localizada em uma única cidade. Ressaltamos que os resultados apresentados são exclusivamente para fins de análise e podem conter vieses, uma vez que refletem apenas a realidade desta oficina específica. A proposta é obter uma visão geral dos veículos que aparecem com maior frequência, além de identificar os maiores valores relacionados a peças, serviços e totais.

In [619]:
# Bibliotecas
import pandas as pd

## Leitura do arquivo e tratamentos iniciais

Fazemos a leitura do arquivo _.csv_ que foi manualmente tratado anteriormente e realizamos a exclusão de colunas que geralmente aparecem.

In [620]:
# Leitura do arquivo tratado manualmente no Excel e remoção de colunas desnecessárias

df = pd.read_csv('ordens-de-servico-por-carro-2024.csv', delimiter=';', index_col=False)
df.drop(columns={df.columns[5], df.columns[6], df.columns[7], df.columns[8]}, inplace=True)

## Transformação de colunas

Sempre gosto de avaliar os dados com a função _.describe()_ do _Pandas_. Para isso, é necessário transformar os valores numéricos em _float_.  
A formatação de moeda para valores em **Real Brasileiro** utiliza-se `.` para casas de milhares e `,` para casas decimais.  
Na leitura do programa **Power BI** que faremos após a execução deste _notebook_, é necessário retornar para o padrão americano para que não haja erro na leitura dos valores. Este passo está realizado no fim do documento.

In [621]:
# Alterando as casas decimais e de milhares para o padrão americano
df.columns = df.columns.str.strip()
df['Produto'] = df['Produto'].str.replace('.', '').str.replace(',', '.').astype(float)
df['Serviço'] = df['Serviço'].str.replace('.', '').str.replace(',', '.').astype(float)
df['Total'] = df['Total'].str.replace('.', '').str.replace(',', '.').astype(float)

# Verificando se a alteração do campo foi realizada
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Data     990 non-null    object 
 1   Veiculo  990 non-null    object 
 2   Produto  990 non-null    float64
 3   Serviço  990 non-null    float64
 4   Total    990 non-null    float64
dtypes: float64(3), object(2)
memory usage: 38.8+ KB


## Removendo veículos menos frequentes

Reconhecemos que neste _dataset_, por ser uma oficina pequena e apenas 1 ano de análise, haverá viéses de todos os tipos. Para que não influencie na leitura dos gráficos demais, carros menos frequentes que raramente comparecem na oficina serão excluídos da análise

In [622]:
# Veículos que aparecem duas ou menos vezes serão desconsiderados 
veiculos_raros = df['Veiculo'].value_counts()
veiculos_raros = veiculos_raros[veiculos_raros <= 2].index

# Alteramos o nome para df_frequentes
df_frequentes = df[~df['Veiculo'].isin(veiculos_raros)]

# Comparando o número de linhas com o df padrão. Anteriormente 990 e agora 858
df_frequentes.describe()

,Produto,Serviço,Total
count,858.000000,858.000000,858.000000
mean,924.221655,643.912436,1568.134091
std,1444.907318,909.080211,2102.158080
min,0.000000,0.000000,5.000000
25%,120.250000,166.102500,387.575000
50%,489.565000,400.000000,896.790000
75%,1179.262500,713.605000,1905.437500
max,17278.710000,8661.280000,21399.990000


## Criação da coluna Fabricante

Este _dataset_ foi adquirido através de um relatório em `.pdf`. Nele, não continha a informação do fabricante do veículo. Item que julgo importante para análise posterior em **Power BI**.  
Com recursos de IA para reconhecimento do nome dos veículos, pude gerar um dicionário com o Fabricante de cada veículo para criação da coluna adicional.

In [623]:
# Dicionário com os fabricantes de cada veículo
fabricantes = {
    'BRAVO': 'FIAT',
    'COROLLA': 'TOYOTA',
    'DUCATO': 'FIAT',
    'S10': 'CHEVROLET',
    '207 SW': 'PEUGEOT',
    'COMPASS': 'JEEP',
    'HRV': 'HONDA',
    'FIESTA': 'FORD',
    'GLA200FF': 'MERCEDES',
    'HB20S': 'HYUNDAI',
    'ONIX': 'CHEVROLET',
    'CELTA': 'CHEVROLET',
    'GRAND SIENA': 'FIAT',
    'HILUX SW4': 'TOYOTA',
    'BMW X1': 'BMW',
    'PAJERO TR4': 'MITSUBISHI',
    'UNO': 'FIAT',
    'KA': 'FORD',
    'RENEGADE': 'JEEP',
    'RAM RAMPAGE': 'RAM',
    'L200 SAVANA': 'MITSUBISHI',
    'MARCH': 'NISSAN',
    'PEUGEOT 208': 'PEUGEOT',
    'L200 TRITON': 'MITSUBISHI',
    'VECTRA': 'CHEVROLET',
    'C4 VTR': 'CITROËN',
    'SANDERO': 'RENAULT',
    'L200 OUTDOOR': 'MITSUBISHI',
    'PAMPA': 'FIAT',
    'HILUX': 'TOYOTA',
    'SPIN': 'CHEVROLET',
    'MERIVA': 'CHEVROLET',
    'PAJERO SPORT': 'MITSUBISHI',
    'CIVIC': 'HONDA',
    'JETTA': 'VOLKSWAGEN',
    'ESCORT': 'FORD',
    'COURIER': 'FORD',
    'LANCER': 'MITSUBISHI',
    'STRADA': 'FIAT',
    'FRONTIER': 'NISSAN',
    'KICKS': 'NISSAN',
    'TRACKER': 'CHEVROLET',
    'ASTRA': 'CHEVROLET',
    'PALIO': 'FIAT',
    'PAJERO': 'MITSUBISHI',
    'PRISMA': 'CHEVROLET',
    'FOX': 'VOLKSWAGEN',
    'C3': 'CITROËN',
    '308': 'PEUGEOT',
    'CRV': 'HONDA',
    'FIORINO': 'FIAT',
    'PALIO WEEKEND': 'FIAT',
    'HB20': 'HYUNDAI',
    'TORO': 'FIAT',
    'PAJERO DAKAR': 'MITSUBISHI',
    'JETTA VARIANT': 'VOLKSWAGEN',
    'MERCEDES CLA': 'MERCEDES',
    'CRUZE': 'CHEVROLET',
    'TIGUAN': 'VOLKSWAGEN',
    'C250': 'MERCEDES',
    'PAJERO GLS-B': 'MITSUBISHI',
    '3008': 'PEUGEOT',
    'ECOSPORT': 'FORD',
    'GRAND CHEROKEE': 'JEEP',
    'GOL': 'VOLKSWAGEN',
    'BMW 320I': 'BMW',
    'C4 CACTUS': 'CITROËN',
    'F250': 'FORD',
    'GALAXIE 500': 'FORD',
    'SIENA': 'FIAT',
    'CORSA': 'CHEVROLET',
    'UP': 'VOLKSWAGEN',
    'FUSION': 'FORD',
    'AZERA': 'HYUNDAI',
    'RANGER': 'FORD',
    'C180': 'MERCEDES',
    'SAVEIRO': 'VOLKSWAGEN',
    'WRV': 'HONDA',
    'CLASSE A160': 'MERCEDES',
    'XC90': 'VOLVO',
    'LIVINA': 'NISSAN',
    'MONTANA': 'CHEVROLET',
    'AMAROK': 'VOLKSWAGEN',
    'BMW 335I': 'BMW',
    'LIFAN X60': 'LIFAN',
    'GOLF': 'VOLKSWAGEN',
    'NEW FIESTA': 'FORD',
    'ARGO': 'FIAT',
    'RAM 2500 LARAMIE': 'RAM',
    'MERCEDES A160': 'MERCEDES',
    'AGILE': 'CHEVROLET',
    'L200': 'MITSUBISHI',
    'XC40': 'VOLVO',
    'SUBARU WRX': 'SUBARU',
    'CAPTUR': 'RENAULT',
    'PICK UP CORSA': 'CHEVROLET',
    'VELOSTER': 'HYUNDAI',
    'MERCEDES B200T': 'MERCEDES',
    'CORSA CLASSIC': 'CHEVROLET',
    'C4 LOUNGE': 'CITROËN',
    'ZAFIRA': 'CHEVROLET',
    'DUSTER': 'RENAULT',
    'FUSCA': 'VOLKSWAGEN',
    'KADETT': 'CHEVROLET',
    'TCROSS': 'VOLKSWAGEN',
    'AUDI Q5': 'AUDI',
    'FASTBACK': 'FIAT',
    'PAJERO FULL': 'MITSUBISHI',
    'DODGE RAM 2500': 'DODGE',
    'MERCEDES CLA45': 'MERCEDES',
    'EXPERT': 'PEUGEOT',
    'ETIOS': 'TOYOTA',
    'GRANDSIENA': 'FIAT',
    'PORSCHE': 'PORSCHE',
    'X1': 'BMW',
    'POLO': 'VOLKSWAGEN',
    'DOBLO': 'FIAT',
    'COBALT': 'CHEVROLET',
    'TUCSON': 'HYUNDAI',
    'FIAT 500': 'FIAT',
    'PASSAT': 'VOLKSWAGEN',
    'MOBI': 'FIAT',
    'PEUGEOT 207': 'PEUGEOT',
    '207': 'PEUGEOT',
    'LINEA': 'FIAT',
    'AUDI RS7': 'AUDI',
    'BMW 330I': 'BMW',
    'VERSA': 'NISSAN',
    'COROLLA FIELDER': 'TOYOTA',
    'SENTRA': 'NISSAN',
    'NOVO CORSA': 'CHEVROLET',
    'DISCOVERY': 'LAND ROVER',
    'MERCEDES': 'MERCEDES',
    'GOLF VARIANT': 'VOLKSWAGEN',
    'BMW X5': 'BMW',
    'ELANTRA': 'HYUNDAI',
    'AUDI A5': 'AUDI',
    'AUDI A3': 'AUDI',
    'BMW 330E': 'BMW',
    'SPACEFOX': 'VOLKSWAGEN',
    'C4 PALLAS': 'CITROËN',
    'CRONOS': 'FIAT',
    'BMW 325I': 'BMW',
    'MERCEDES C250': 'MERCEDES',
    'MASTER': 'RENAULT',
    'CITY': 'HONDA',
    'ACCORD': 'HONDA',
    'DISCOVERY IV': 'LAND ROVER',
    'IX35': 'HYUNDAI',
    'ONIX PLUS': 'CHEVROLET',
    'SUZUKI SX4': 'SUZUKI',
    'ASX': 'MITSUBISHI',
    'SANTA FE': 'HYUNDAI',
    'PARATI': 'VOLKSWAGEN',
    'FUSCA NEW': 'VOLKSWAGEN',
    'FLUENCE': 'RENAULT',
    'BMW X6': 'BMW',
    'DISCOVERY 4': 'LAND ROVER',
    'VERA CRUZ': 'HYUNDAI',
    'TRAILBLAZER': 'CHEVROLET',
    'HUILUX': 'TOYOTA',
    'C200': 'MERCEDES',
    'PREMIO': 'TOYOTA',
    'BMW 528I': 'BMW',
    'CHEVETTE': 'CHEVROLET',
    'I30': 'HYUNDAI',
    'NIVUS': 'VOLKSWAGEN',
    'AUDI 80': 'AUDI',
    'OMEGA': 'CHEVROLET',
    'BONGO': 'KIA',
    'VOLVO XC60': 'VOLVO',
    'OUTLANDER': 'MITSUBISHI',
    'VIRTUS': 'VOLKSWAGEN',
    'CAPTIVA': 'CHEVROLET',
    'HB20X': 'HYUNDAI',
    'AUDI Q3': 'AUDI',
    'TROLLER T4': 'TROLLER',
    'FIT': 'HONDA',
    'X4': 'BMW',
    'LAND CRUISER': 'TOYOTA',
    'OPALA': 'CHEVROLET',
    'VOYAGE': 'VOLKSWAGEN',
    'T-CROSS': 'VOLKSWAGEN',
    'VOLVO C30': 'VOLVO',
    'MINI COOPER S': 'MINI',
    'UNO MILLE': 'FIAT',
    'MERCEDES B200': 'MERCEDES',
    'VITARA': 'SUZUKI',
    'SONATA': 'HYUNDAI'
}

# Criação da coluna Fabricante no df_frequentes utilizando a função .map que analisa cada veículo e acrescenta o fabricante subsequente
df_frequentes['Fabricante'] = df_frequentes['Veiculo'].map(fabricantes)

# Visualização da coluna Fabricante
df_frequentes.head()

C:\Users\flavianojr1\AppData\Local\Temp\ipykernel_40140\476453425.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_frequentes['Fabricante'] = df_frequentes['Veiculo'].map(fabricantes)


,Data,Veiculo,Produto,Serviço,Total,Fabricante
1,02/01/2024,COROLLA,0.00,1100.00,1100.00,TOYOTA
3,02/01/2024,S10,1650.77,1237.18,2887.95,CHEVROLET
4,02/01/2024,207 SW,382.50,945.00,1327.50,PEUGEOT
5,03/01/2024,COMPASS,1104.35,325.00,1429.35,JEEP
6,03/01/2024,HRV,0.00,2090.00,2090.00,HONDA


## Fabricantes redudantes

No _dataset_ de importação, alguns veículos vieram com o fabricante denominado anteriormente ao nome do veículo, como por exemplo `BMW X1`.  
Nomes como `208` ou `A5` que são mais curtos, também apareceram em alguns dos casos.  
No código abaixo alteramos essa exibição para constar apenas o nome do veículo e deixar a coluna Fabricante fazer o seu trabalho.

In [624]:
# Alguns casos de Fabricantes que vieram acompanhados do nome do veículo são removidos pela função str.replace()
df_frequentes['Veiculo'] = df_frequentes['Veiculo'].str.replace('BMW ', '')
df_frequentes['Veiculo'] = df_frequentes['Veiculo'].str.replace('PEUGEOT ', '')
df_frequentes['Veiculo'] = df_frequentes['Veiculo'].str.replace('MERCEDES ', '')
df_frequentes['Veiculo'] = df_frequentes['Veiculo'].str.replace('AUDI ', '')

C:\Users\flavianojr1\AppData\Local\Temp\ipykernel_40140\3365790863.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_frequentes['Veiculo'] = df_frequentes['Veiculo'].str.replace('BMW ', '')
C:\Users\flavianojr1\AppData\Local\Temp\ipykernel_40140\3365790863.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_frequentes['Veiculo'] = df_frequentes['Veiculo'].str.replace('PEUGEOT ', '')
C:\Users\flavianojr1\AppData\Local\Temp\ipykernel_40140\3365790863.py:4: SettingWithCopyWarning: 
A value is trying to

## Alteração das casas decimais

Como mencionado mais acima, o padrão de leitura de casas decimais entre o padrão americano e brasileiro têm suas divergências.  
Aqui alteramos novamente ao estado original para que a leitura seja realizada sem erros no **Power BI**.

In [625]:
# Alteração das casas decimais para valores monetários
df_frequentes['Produto'] = df_frequentes['Produto'].astype(str).str.replace('.', ',') 
df_frequentes['Serviço'] = df_frequentes['Serviço'].astype(str).str.replace('.', ',')
df_frequentes['Total'] = df_frequentes['Total'].astype(str).str.replace('.', ',')

# Exibição do dataframe para visualização
# df_frequentes.head()

# Salvamos o arquivo tratado em .csv para leitura no Power BI
df_frequentes.to_csv('ordens-de-servico-por-carro-2024-tratado.csv', sep=',', index=False)

C:\Users\flavianojr1\AppData\Local\Temp\ipykernel_40140\1757986025.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_frequentes['Produto'] = df_frequentes['Produto'].astype(str).str.replace('.', ',')
C:\Users\flavianojr1\AppData\Local\Temp\ipykernel_40140\1757986025.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_frequentes['Serviço'] = df_frequentes['Serviço'].astype(str).str.replace('.', ',')
C:\Users\flavianojr1\AppData\Local\Temp\ipykernel_40140\1757986025.py:4: SettingWithCopyWarning: 
A va